# Step 2. Model Traning

> Basically It's highly recommended that following DLI turorial first
> During your own dataset training, modify below parts for your perference.

###  Traning epochs and semantic class numbers

In [ ]:
epochs = 15
num_classes = 6

### Directory Paths

In [ ]:
data_dir = "/home/kimsooyoung/Documents/grocery_data_2024-05-23_16:43:00"
output_file = "/home/kimsooyoung/Documents/model.pth"

### Static Labels

In [ ]:
# Labels for the dataset
static_labels = {
    'apple' : 0,
    'avocado' : 1,
    'kiwi' : 2,
    'lime' : 3,
    'lychee' : 4,
    'pomegranate' : 5,
    'onion' : 6,
    'strawberry' : 7,
    'lemon' : 8,
    'orange' : 9
}

### Training Batch Size

In [ ]:
# create our dataset by using our custom GroceryDataset class
# This is then passed into our DataLoader.
dataset = FruitDataset(data_dir, get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=16, # 8 or 4
    shuffle=True,
    collate_fn= collate_fn
)

## Step 3. Model Export

> Basically It's highly recommended that following DLI turorial first
> During your own dataset training, modify below parts for your perference.

if your camera resolution was (1280, 720)

```python
self.rp = rep.create.render_product(self.cam, resolution=(1280, 720))
```

you must modify `dummy_input` as belows


```python
# Export Model
width = 1280
height = 720
dummy_input = torch.rand(1, 3, height, width).cuda()

```

## Step 4. Model Inference

> This example will use [NVIDIA's Triton Inference Server](https://developer.nvidia.com/nvidia-triton-inference-server) to serve the model exported in the previous section.

- Select one pic from dataset or make new own from replicator
- Run Inference with trained models and save the output
- Vizualize output    

![](https://www.notion.so/image/https%3A%2F%2Fprod-files-secure.s3.us-west-2.amazonaws.com%2F5fbee23e-d9ec-4824-b2be-c8716a8602cd%2Ffb1d2d9b-ec88-44f0-9674-ce696073a4ad%2FUntitled.png?table=block&id=11da0ea2-0123-4a1c-a549-4987a6c471d9&spaceId=5fbee23e-d9ec-4824-b2be-c8716a8602cd&width=2000&userId=b9f59011-0253-43ef-946a-512501504ba8&cache=v2)


### NGC Triton Docker example

- Assume that you made and saved model into `/home/<my-pc-name>/Documents/model_repository/`
- You can pull and run docker image like this


```
docker run --gpus=1 --rm -p8000:8000 -p8003:8001 -p8004:8002 -v /home/<my-pc-name>/Documents/model_repository/:/models nvcr.io/nvidia/tritonserver:23.01-py3 tritonserver --model-repository=/models --model-control-mode=poll
```



> check this terminal log for successful import

![](https://www.notion.so/image/https%3A%2F%2Fprod-files-secure.s3.us-west-2.amazonaws.com%2F5fbee23e-d9ec-4824-b2be-c8716a8602cd%2Fcf09d2ed-5519-40c6-b816-af4e984c4b0d%2FUntitled.png?table=block&id=9cd9b07f-374e-4f51-9b2d-684e7c1abd17&spaceId=5fbee23e-d9ec-4824-b2be-c8716a8602cd&width=2000&userId=b9f59011-0253-43ef-946a-512501504ba8&cache=v2)

# Instructor Demonstration - Conveyor Belt in Grocery Factory

![](https://www.notion.so/image/https%3A%2F%2Fprod-files-secure.s3.us-west-2.amazonaws.com%2F5fbee23e-d9ec-4824-b2be-c8716a8602cd%2Ff0eebfe2-24b2-4be0-b64b-b8fe754c3047%2FScreenshot_from_2024-07-31_00-11-59.png?table=block&id=f7274350-b8de-4743-8dda-bcf68a31f410&spaceId=5fbee23e-d9ec-4824-b2be-c8716a8602cd&width=2000&userId=b9f59011-0253-43ef-946a-512501504ba8&cache=v2)

> If you want to add annotation labels like instructor...

![](https://www.notion.so/image/https%3A%2F%2Fprod-files-secure.s3.us-west-2.amazonaws.com%2F5fbee23e-d9ec-4824-b2be-c8716a8602cd%2F68ee5a1f-4bb7-4fce-b86a-3f3652410e3a%2Fannotated_img.png?table=block&id=6b7a9974-b76e-4ed8-bee5-7bf0e22a9a9a&spaceId=5fbee23e-d9ec-4824-b2be-c8716a8602cd&width=2000&userId=b9f59011-0253-43ef-946a-512501504ba8&cache=v2)

1. Define `props_dict` like below

```python
props_dict = {
    0: 'klt_bin',
    1: 'tomato_soup',
    2: 'tuna',
    3: 'spam',
    4: 'jelly',
    5: 'cleanser',
}
```

2. Modify viz function like below

```python
if boxes.size > 0:  # ensure something is found
    for box, lab, scr in zip(boxes, labels, scores):
        if scr > 0.4:
            box_top_left = int(box[0]), int(box[1])
            box_bottom_right = int(box[2]), int(box[3])
            text_origin = int(box[0]), int(box[3])

            border_color = list(np.random.random(size=3) * 256)
            text_color = (255, 255, 255)

            font_scale = 0.9
            thickness = 1

            # bounding box2
            img = cv2.rectangle(
                annotated_image,
                box_top_left,
                box_bottom_right,
                border_color,
                thickness=5,
                lineType=cv2.LINE_8
            )

            print(f"index: {lab}, label: {props_dict[lab]}, score: {scr:.2f}")

            # For the text background
            # Finds space required by the text so that we can put a background with that amount of width.
            (w, h), _ = cv2.getTextSize(
                props_dict[lab], cv2.FONT_HERSHEY_SIMPLEX,
                0.6, 1
            )

            # Prints the text.    
            img = cv2.rectangle(
                img, (box_top_left[0], box_top_left[1] - 20),
                (box_top_left[0] + w, box_top_left[1]),
                border_color, -1
            )
            img = cv2.putText(
                img, props_dict[lab], box_top_left,
                cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                text_color, 1
            )
    
```